# For PLOS ONE Revision 2023-06

## SETUP

In [24]:
# Import libraries
import numpy as np
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm
import random
from scipy.stats.contingency import chi2_contingency


pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

In [9]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC-III Text Mining/LOS_FINAL/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()

  
print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"

/Users/ADORNI/Dropbox (BFI)/Luca_Data_df_mixed/


## LOAD DATA

In [11]:
# PARAMETERS

# Set to True if we want to include deaths
death_incl = False
death_tag = np.where(death_incl,"_death", "")

In [14]:
# import dataset
file = f'{path_to_processed}df_mixed_discharge{death_tag}.csv.gzip'
df = pd.read_csv(file, compression = 'gzip', low_memory=False)

In [15]:
# Transform the LOS timedelta to days (as decimal values to increase precision)
df['los'] = pd.to_timedelta(df.los)
df['los'] = df.los/pd.to_timedelta(1, unit='D')

In [16]:
# selection criterion : only patients 18 and older and with a length of stay or 1 day or greater
df = df.loc[(df['age']>=18) & (df['los']>=1),:]
# compare size of dataframe before and after selection
len(raw_df), len(df)

(30985, 30764)

In [17]:
# drop the variables to be exempted from the analysis and rename new dataset
df = df.drop(columns = ['hadm_id', 'subject_id','icu_los'])

In [18]:
# check proportion of missing values
missing = pd.concat([df.isna().sum(), df.isna().mean()], axis = 1)
missing.columns = ['count', 'proportions']
missing.sort_values('proportions',ascending = False)

,count,proportions
patientweight,14847,0.482610
albumin_min,10503,0.341406
calcium_min,819,0.026622
glucose_mean,64,0.002080
glucose_max,64,0.002080
glucose_min,64,0.002080
magnesium_max,64,0.002080
temp_mean,45,0.001463
temp_max,45,0.001463
temp_min,45,0.001463


In [19]:
# drop variables having more than 20 % missing values
print(f"Variables with more than 20% Missing Values: {list(missing.loc[missing.proportions >= 0.2].index)}")
df = df.drop(columns=missing.loc[missing.proportions >= 0.2].index)

Variables with more than 20% Missing Values: ['albumin_min', 'patientweight']


In [20]:
# impute missing values
df = df.interpolate()
df.isna().mean()

ethnicity             0.0
admission_type        0.0
admission_location    0.0
insurance             0.0
religion              0.0
marital_status        0.0
gender                0.0
age                   0.0
urea_n_min            0.0
urea_n_max            0.0
urea_n_mean           0.0
platelets_min         0.0
platelets_max         0.0
platelets_mean        0.0
magnesium_max         0.0
calcium_min           0.0
resprate_min          0.0
resprate_max          0.0
resprate_mean         0.0
glucose_min           0.0
glucose_max           0.0
glucose_mean          0.0
hr_min                0.0
hr_max                0.0
hr_mean               0.0
sysbp_min             0.0
sysbp_max             0.0
sysbp_mean            0.0
diasbp_min            0.0
diasbp_max            0.0
diasbp_mean           0.0
temp_min              0.0
temp_max              0.0
temp_mean             0.0
sapsii                0.0
sofa                  0.0
urine_min             0.0
urine_mean            0.0
urine_max   

In [21]:
# compute Lower and Upper Fence according to Tukey's criteria
y = df['los']
Q1 = np.percentile(y, 25)
Q3 = np.percentile(y, 75)
IQR = Q3-Q1
LF = Q1 - 1.5*IQR
UF = Q3 + 1.5*IQR
print(f'First quartile = {Q1:.3f}, Third Quartile = {Q3:.3f}, Interquartile Interval = {IQR:.3f}')
print(f'Lower Fence = {LF:.3f}, Upper Fence = {UF:.3f}')

First quartile = 5.051, Third Quartile = 13.388, Interquartile Interval = 8.338
Lower Fence = -7.456, Upper Fence = 25.895


In [22]:
# create categorical LOS variable where prolonged LOS is any value greater than Upper Fence
df['los_cat'] = df['los']> UF
df = df.drop(columns=['los'])

In [23]:
# Display the proportions
df['los_cat'].value_counts()

False    28525
True      2239
Name: los_cat, dtype: int64

## Exploring PLOS for Age Category and other variables

In [26]:
# Crosstab and explore the Age Category/Admission Type
ct1_obs = pd.crosstab(df['age_cat'], df['admission_type'])
chi2, p_value, dof, ct1_exp = chi2_contingency(ct1_obs)

In [81]:
def chi_square (raw1, raw2) :
  '''
  returns chi-square statistics, p-value, degree of freedom, observed, expected, and residual contingency table respectively
  '''
  # Perform the crosstab between the two variables
  ct_obs = pd.crosstab(raw1,raw2)
  # Calculate the Chi-square test
  Khi_square, p_value, dof, ct_exp = chi2_contingency(ct_obs)
  ct_res = (ct_obs.values - ct_exp)/np.sqrt(ct_exp)

  # Perform the crosstab between the two variables
  ct_obs = pd.crosstab(raw1,raw2)
  # Calculate the Chi-square test
  Khi_square, p_value, dof, ct_exp = chi2_contingency(ct_obs)
  ct_res = (ct_obs.values - ct_exp)/np.sqrt(ct_exp)

  # Include the expected values
  for c, _ in enumerate(ct_obs.columns):
      for r, _ in enumerate(ct_obs.index):
          ct_obs.iloc[r,c] = str(ct_obs.iloc[r,c]) + " [" + str(int(ct_exp[r,c])) + "]"


  # Include the statistics
  bottom_stats = [round(Khi_square,3)] + ["" for i in enumerate(ct_obs.columns[1:])]
  bottom_stats2 = [round(p_value,5)] + ["" for i in enumerate(ct_obs.columns[1:])]
  ct_obs.loc['Chi Square'] = bottom_stats
  ct_obs.loc['p-value'] = bottom_stats2

  return ct_obs

In [82]:
age_res = chi_square(df['age_cat'], df['admission_type'])
age_res.to_csv(path_to_repo + f"/tables/chi2_age_admission{death_tag}.csv")
age_res

admission_type,ELECTIVE,EMERGENCY,URGENT
age_cat,,,
18-44 years,394 [636],3532 [3305],81 [65]
45-64 years,1953 [1668],8394 [8673],168 [172]
65-84 years,2311 [2063],10474 [10723],215 [212]
85+ years,225 [514],2977 [2674],40 [53]
Chi Square,404.679,,
p-value,0.0,,


In [83]:
adm_type = chi_square(df['admission_type'], df['los_cat'])
adm_type.to_csv(path_to_repo + f"/tables/chi2_los_admission{death_tag}.csv")
adm_type

los_cat,False,True
admission_type,,
ELECTIVE,4654 [4527],229 [355]
EMERGENCY,23435 [23530],1942 [1846]
URGENT,436 [467],68 [36]
Chi Square,82.59,
p-value,0.0,


In [84]:
age_los = chi_square(df['age_cat'], df['los_cat'])
age_los.to_csv(path_to_repo + f"/tables/chi2_los_age{death_tag}.csv")
age_los

los_cat,False,True
age_cat,,
18-44 years,3629 [3715],378 [291]
45-64 years,9573 [9749],942 [765]
65-84 years,12186 [12053],814 [946]
85+ years,3137 [3006],105 [235]
Chi Square,169.885,
p-value,0.0,


In [85]:
emg = chi_square(df['age_cat'], df['emergency_dpt'])
emg.to_csv(path_to_repo + f"/tables/chi2_dpt_age{death_tag}.csv")
emg

emergency_dpt,No,Yes
age_cat,,
18-44 years,394 [636],3613 [3370]
45-64 years,1953 [1668],8562 [8846]
65-84 years,2311 [2063],10689 [10936]
85+ years,225 [514],3017 [2727]
Chi Square,395.934,
p-value,0.0,


In [86]:
stay = chi_square(df['age_cat'], df['type_stay'])
stay.to_csv(path_to_repo + f"/tables/chi2_stay_age{death_tag}.csv")
stay

type_stay,1-Medical,2-Obstetrics,3-Surgical
age_cat,,,
18-44 years,2646 [2342],41 [5],1320 [1658]
45-64 years,5695 [6148],2 [15],4818 [4351]
65-84 years,7156 [7601],1 [18],5843 [5380]
85+ years,2491 [1895],0 [4],751 [1341]
Chi Square,954.205,,
p-value,0.0,,


In [87]:
prev_adm = chi_square(df['age_cat'], df['type_stay'])
prev_adm.to_csv(path_to_repo + f"/tables/chi2_prev_age{death_tag}.csv")
prev_adm

type_stay,1-Medical,2-Obstetrics,3-Surgical
age_cat,,,
18-44 years,2646 [2342],41 [5],1320 [1658]
45-64 years,5695 [6148],2 [15],4818 [4351]
65-84 years,7156 [7601],1 [18],5843 [5380]
85+ years,2491 [1895],0 [4],751 [1341]
Chi Square,954.205,,
p-value,0.0,,
